In [1]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [2]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

c:\Users\riahl\OneDrive\UKCareer\Careers\Resume\Companies\PrynGlobal\FoundationsMVPBuild\AzureDevops\ai-agents-for-beginners\Agentsenv\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


In [3]:
frontdesk_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.""",
)

concierge_agent = AssistantAgent(
    "concierge_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, respond with 'APPROVE'
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """,
)

In [4]:
termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat(
    [frontdesk_agent, concierge_agent], termination_condition=termination
)

async for message in team.run_stream(task="I would like to plan a trip to Paris."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='b0fe80dd-5a5b-46aa-a444-7337c72d6c0c' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 19, 18, 44, 30, 291517, tzinfo=datetime.timezone.utc) content='I would like to plan a trip to Paris.' type='TextMessage'
id='0e3aaba0-f7ab-428e-95fc-24cd881c68b6' source='planner_agent' models_usage=RequestUsage(prompt_tokens=100, completion_tokens=11) metadata={} created_at=datetime.datetime(2025, 7, 19, 18, 44, 31, 197363, tzinfo=datetime.timezone.utc) content='Visit the Louvre Museum for an iconic cultural experience.' type='TextMessage'
id='45dbe13f-9a33-4254-a9e4-bb5775136c37' source='concierge_agent' models_usage=RequestUsage(prompt_tokens=112, completion_tokens=59) metadata={} created_at=datetime.datetime(2025, 7, 19, 18, 44, 33, 320186, tzinfo=datetime.timezone.utc) content='While the Louvre Museum is indeed an iconic destination, it can be quite touristy and crowded. To refine the recommendation, consider suggesting a lesser-known museum or art space that